In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from tensorflow.keras.datasets import fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images.shape

4423680/4422102 [==============================] - 0s 0us/step


(60000, 28, 28)

In [2]:
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Sequential

model = Sequential([
    Flatten(input_shape=(28,28)),
    Dense(100, activation='relu'),
    Dense(30, activation='relu'),
    Dense(10, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                310       
Total params: 81,840
Trainable params: 81,840
Non-trainable params: 0
_________________________________________________________________


### categorical crossentropy를 적용하려면 target값은 반드시 ```one hot encoding``` 해야된다
one hot encoding 안할거면 sparse_categorical_crossentropy 사용

In [5]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy

model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
from tensorflow.keras.utils import to_categorical

train_oh_labels = to_categorical(train_labels)
test_oh_labels = to_categorical(test_labels)

history = model.fit(x=train_images, y=train_oh_labels, batch_size=32, epochs=20, verbose=1)
print(history.history['loss'])
print(history.history['accuracy'])

Epoch 1/20
1875/1875 [==============================] - 4s 2ms/step - loss: 5.3910 - accuracy: 0.5273
Epoch 2/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.7798 - accuracy: 0.6897
Epoch 3/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.6975 - accuracy: 0.7199
Epoch 4/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.6243 - accuracy: 0.7461
Epoch 5/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5564 - accuracy: 0.7820
Epoch 6/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4774 - accuracy: 0.8287
Epoch 7/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4325 - accuracy: 0.8462
Epoch 8/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4084 - accuracy: 0.8559
Epoch 9/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4006 - accuracy: 0.8570
Epoch 10/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.381

In [8]:
pred_proba = model.predict(test_images)
print(pred_proba.shape)

(10000, 10)


전체 데이터셋을 predict할 경우 위의 cell처럼 n개의 28 * 28개의 데이터를 넣으면 되는데 하나의 이미지만 predict 시킬 때는 차원을 바꿔줘야한다

```(10000,28,28)```형태로 3차원이어야 하는데 test_images[0]으로 하나의 이미지만 불러오면 ```(28,28)``` 이런식이기 때문

```np.expand_dims```을 이용하여 차원을 늘려준다( []를 한겹 더 씌워준다고 생각하면됨)

In [9]:
test_images[0].shape

(28, 28)

predict하여 각각의 class값에 대해 softmax 값을 구함

np.argmax로 최대값을 고른다

np.squeeze를 하면 predict를 할때 expand_dims로 차원을 늘린것을 다시 축소시켜준다

In [12]:
pred_proba = model.predict(np.expand_dims(test_images[0], axis=0))
print('softmax output : ', pred_proba)
pred = np.argmax(np.squeeze(pred_proba))
print('predicted class value : ', pred)

softmax output :  [[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  7.2401721e-04 0.0000000e+00 4.6238527e-02 4.8881359e-12 9.5303744e-01]]
predicted class value :  9


In [13]:
model.evaluate(test_images, test_oh_labels, batch_size=64)

157/157 [==============================] - 0s 2ms/step - loss: 0.4430 - accuracy: 0.8581


[0.4430147111415863, 0.8580999970436096]